In [7]:
import numpy as np
import pandas as pd
import gdown
import warnings

warnings.filterwarnings('ignore')

# Data Preparation

In [8]:
url = "https://drive.google.com/file/d/1WgdS7nR_z9iHFoFd38UfxxBzo5HkjslL"

gdown.download(url, "Reviews.csv", quiet=False)

df = pd.read_csv("Reviews.csv")
df.sample(5)

Downloading...
From: https://drive.google.com/file/d/1WgdS7nR_z9iHFoFd38UfxxBzo5HkjslL
To: c:\Users\Devve\Documents\Machine learning\gift-recommendation-system\notebook\Reviews.csv
8.91kB [00:00, 348kB/s]


,"<!DOCTYPE html><style nonce=""6s8fAb55RZP53c1rMfPHlQ"">body{height:100%;margin:0;width:100%}@media (max-height:350px){.button{font-size:10px}.button-container{margin-top:16px}.button.primary-button",.button.primary-button:active,.button.primary-button:focus,.button.primary-button:hover{padding:4px 12px}.title-text{font-size:22px;line-height:24px}.subtitle-text{font-size:12px;line-height:18px}}@media (min-height:350px){.button{font-size:14px}.button-container{margin-top:16px}.button.primary-button,.button.primary-button:active.1,.button.primary-button:focus.1,.button.primary-button:hover{padding:12px 24px}.title-text{font-size:28px;line-height:36px}.subtitle-text{font-size:16px;line-height:24px}}.document-root{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex;inset:0;position:absolute}.error,.login,.request-storage-access{display:none}.error,.login.1,...,0.1,.149),0 1px 2px 0 rgba(0,0.2,0.3,.302)}.icon{height:48px;margin-bottom:16px}.title-text{font-family:Google Sans,Roboto.1,sans-serif;text-align:center}.subtitle-text{font-family:Google Sans Text,Roboto.2,sans-serif;margin-top:16px;text-align:center}
11,function u(a,b){var c;if(b=(c=b.currentTarget)==null?void 0...,B(b)||D);F(a.l,function(){d&&d.close();var e=window.location....,"g=1;f.has(""login_counter"")&&(f=Number(f.get(""l...",isFinite(f)&&(g=f+1));e=m(e,login_counter,String(g));A(a,e)})}};function G(a){this.g=a}G.prototype.toSt...,"a.length+1).toLowerCase()===a+"":""})}var J=[I(""...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,function C(a){if(a instanceof G)if(a instanceo...,b){b=C(b);return b!==void 0?a.open(b,popupWindow,"popup=yes,height=500,width=690):null};function...",c=b.next();!c.done;c=b.next())if(c=c.value.spl...,c[0].trim()===a)return c[1]};function q(){this...,__Secure-1PAPISID,__Secure-3PAPISID];this.g=void 0}function F(a,b){a.g&&clearInterval(a.g);for(var c={},d=k(a.h),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,"</script><div class=""document-root loading""><d...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,*/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,function x(){if(window.navigator.userAgentData...,"b=0;b<a.length;b++){var c=a[b];if(c.brand===""G...",M);function M(){v()};,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

In [ ]:
df.dropna(inplace= True)
df.info()

In [ ]:
df = df[df['HelpfulnessNumerator'] <= df['HelpfulnessDenominator']]

# Data Aggregation

In [ ]:
# Group by ProductId and apply aggregation
grouped_df = df.groupby('ProductId').agg(
    Summary_Text=('Summary', ' '.join),  # Merge Summary and Text
    Score=('Score', 'mean'),  # Calculate the average score
    HelpfulnessDenominator = ('HelpfulnessDenominator','sum'),
    HelpfulnessNumerator = ('HelpfulnessNumerator','sum')
).reset_index()

# Merge the 'Text' column as well to 'Summary_Text'
grouped_df['Summary_Text'] = grouped_df['Summary_Text'] + ' ' + df.groupby('ProductId')['Text'].agg(' '.join).reset_index()['Text']

# Display the first few rows of the grouped dataframe
grouped_df.sample(5)

In [ ]:
grouped_df.info()

In [ ]:
import string
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from stopwords import get_stopwords

stemmer = SnowballStemmer("english")
stop_words = set(get_stopwords('en'))
punctuations = string.punctuation

def cleanText(text):
    tokens = word_tokenize(text)

    new_tokens = [stemmer.stem(word.lower()) for word in tokens if word not in stop_words and word not in punctuations]

    return " ".join(new_tokens)

grouped_df['Summary_Text'] = grouped_df['Summary_Text'].apply(cleanText)


In [ ]:
grouped_df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=10000,  # Limit to top 10,000 features
    stop_words='english',  # Remove English stopwords
    max_df=0.8,  # Ignore terms that appear in more than 80% of documents
    min_df=5     # Ignore terms that appear in fewer than 5 documents
)
vectors = vectorizer.fit_transform(grouped_df['Summary_Text'])
print(vectors.shape)